## Example of a classifier

In [27]:
# EDA and data handling
import numpy as np 
import pandas as pd 
import pickle

# Modeling
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics

In [4]:
# read in the data
# housing_train = pd.read_csv('/content/sample_data/california_housing_train.csv')
housing_train = pd.read_csv('california_housing_train.csv')
housing_train.shape

(17000, 9)

In [6]:
# reduce the size of the dataset
housing_train = housing_train.sample(500)

In [7]:
# show the data 
housing_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
14199,-122.08,37.84,17.0,1320.0,159.0,1722.0,141.0,11.7064,500001.0
15813,-122.41,37.70,23.0,1817.0,400.0,1376.0,382.0,2.4113,214200.0
6457,-118.27,34.10,41.0,3729.0,740.0,1364.0,707.0,5.7778,412700.0
14024,-122.04,36.96,32.0,1438.0,306.0,802.0,293.0,4.1964,202000.0
13955,-122.04,38.27,16.0,8517.0,1910.0,4508.0,1837.0,3.1853,129600.0


In [8]:
# Describe the target 
housing_train['median_house_value'].describe()

count       500.000000
mean     198419.628000
std      110954.431349
min       43600.000000
25%      116275.000000
50%      172650.000000
75%      258875.000000
max      500001.000000
Name: median_house_value, dtype: float64

In [9]:
# create the target 
housing_train['high_price']=np.where(housing_train['median_house_value']>=187250, 1, 0)

In [21]:
# establish the predictors and the target
X = housing_train.drop(['median_house_value','high_price'], axis=1)
y = housing_train['high_price']

In [22]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.3, random_state=42 )
print('length of y-test:', len(y_test))

length of y-test: 150


In [25]:
# instantiate the classifier
mymodel = RandomForestClassifier()

In [26]:
# fit on the training data
mymodel.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
# predict on the testing data
y_preds = mymodel.predict(X_test)

accuracy score:  0.8


In [85]:
# check out the first few houses
print(y_test.values[:10], 'true')
print(y_preds[:10], 'predicted')

[0 1 1 1 1 0 0 1 0 1] true
[0 1 1 1 0 0 0 1 0 1] predicted


In [ ]:
# evaluate the model performance
print('accuracy score: ', round(metrics.accuracy_score(y_test, y_preds),2))

## Confusion Matrix

In [51]:
# examine the confusion matrix
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_preds), columns=['pred_0', 'pred_1'])
cm

,pred_0,pred_1
0,72,15
1,15,48


* 48 true positives (TP): These are cases in which we predicted yes (pricey house), and it is indeed a pricey house.
* 72 true negatives (TN): We predicted no (cheap house), and the house is indeed cheap.
* 15 false positives (FP): We predicted yes (pricey house), but the house isn't pricey. (Also known as a "Type I error.")
* 15 false negatives (FN): We predicted no (cheap house), but the house is actually pricey. (Also known as a "Type II error.")

<div>
<img src="https://miro.medium.com/max/1400/1*h1EA_HjN0jSUh1y6SxdTKQ.png" width="500"/>
</div>
SOURCE: https://towardsdatascience.com/machine-learning-an-error-by-any-other-name-a7760a702c4d

In [65]:
# get the numbers
TN=cm['pred_0'].values[0]
FN=cm['pred_0'].values[1]
FP=cm['pred_1'].values[0]
TP=cm['pred_1'].values[1]
ALL=cm.values.sum()

In [78]:
print('True Negatives:', TN)
print('False Negatives:', FN)
print('False Positives:', FP)
print('True Positives:', TP)
print('All:', ALL)

True Negatives: 72
False Negatives: 15
False Positives: 15
True Positives: 48
All: 150


**Accuracy:**  
Overall, how often is it correct?

In [68]:
print(f'Accuracy: {round((TP + TN)/ALL, 2)}')

Accuracy: 0.8


**True Positive Rate:**   
When it's actually yes, how often does it predict yes?        
“Sensitivity” or “Recall”

In [101]:
print(f'True Positive Rate: {round(TP/ (TP + FN), 2)}')

True Positive Rate: 0.76


**False Positive Rate:**    
When it's actually no, how often does it predict yes?   
Also known as "Fall-out Rate"

In [74]:
print(f'False Positive Rate: {round(FP / (TN + FP), 2)}')

False Positive Rate: 0.17


**Precision:**     
When it predicts yes, how often is it correct?  
Also known as "Positive Predictive Value (PPV)"

In [70]:
print(f'Precision: {round(TP / (TP + FP), 2)}')

Precision: 0.76


**Specificity:**  
When it's actually no, how often does it predict no?    
also known as "True Negative Rate"

In [71]:
print(f'Specificity: {round(TN / (TN + FP), 2)}')

Specificity: 0.83


**Prevalence:**    
How often does the yes condition actually occur in our sample?  
actual yes/total

In [62]:
print(f'Prevalence: {round(TP + FN / ALL, 2)}')

Prevalence: 72.1


## Again

In [88]:
# House age
housing_train['housing_median_age'].describe()

count    500.000000
mean      27.894000
std       12.493476
min        2.000000
25%       17.000000
50%       28.000000
75%       36.000000
max       52.000000
Name: housing_median_age, dtype: float64

In [89]:
# Create a target
housing_train['old']=np.where(housing_train['housing_median_age']>=28, 1, 0)

In [90]:
# establish the predictors and the target
X = housing_train.drop(['housing_median_age','high_price', 'old'], axis=1)
y = housing_train['old']

In [91]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.3, random_state=42 )
print('length of y-test:', len(y_test))

length of y-test: 150


In [92]:
# instantiate the classifier
mymodel = RandomForestClassifier()

In [93]:
# fit on the training data
mymodel.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [94]:
# predict on the testing data
y_preds = mymodel.predict(X_test)

In [95]:
# check out the first few houses
print(y_test.values[:10], 'true')
print(y_preds[:10], 'predicted')

[1 1 1 0 1 1 1 0 1 0] true
[1 1 1 0 1 0 1 0 1 0] predicted


In [96]:
# evaluate the model performance
print('accuracy score: ', round(metrics.accuracy_score(y_test, y_preds),2))

accuracy score:  0.73


In [97]:
# examine the confusion matrix
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_preds), columns=['pred_0', 'pred_1'])
cm

,pred_0,pred_1
0,60,24
1,16,50


In [98]:
# get the numbers
TN=cm['pred_0'].values[0]
FN=cm['pred_0'].values[1]
FP=cm['pred_1'].values[0]
TP=cm['pred_1'].values[1]
ALL=cm.values.sum()

In [100]:
print(f'Accuracy: {round((TP + TN)/ALL, 2)}')
print(f'True Positive Rate: {round(TP/ (TP + FN), 2)}')
print(f'False Positive Rate: {round(FP / (TN + FP), 2)}')
print(f'Precision: {round(TP / (TP + FP), 2)}')
print(f'Specificity: {round(TN / (TN + FP), 2)}')
print(f'Prevalence: {round(TP + FN / ALL, 2)}')

Accuracy: 0.73
True Positive Rate: 0.76
False Positive Rate: 0.29
Precision: 0.68
Specificity: 0.71
Prevalence: 50.11


## Terminology

| Bayes term | Bayes formula | Confusion Matrix term | Confusion Matrix formula| Alternative CM term | 
|:-|:-|:-|:-|:-|
| prior | P(A) | prevalence | (TP + FN) / (TP+TN+FP+FN) | ? |
| posterior | P(A given B) | Positive Predictive Value (PPV) | TP / (TP + FP) | precision |
| conditional | P(B given A) | True Positive Rate (TPR)  |TP / (TP + FN) | sensitivity, recall |
| marginal | P(B) | queue rate | TP + FP | ? |  
| prior complement | P(not A) or 100-P(A) | prevalence complement | 1-prevalence | ? |
| ? | P(not B given not A) | True Negative Rate (TNR) | TN / (FP + TN) | specificity |
| ? | P(B given not A) | False Positive Rate (FPR) | FP / (FP+TN) | fall-out rate, false alarm rate |
| ? | P(not B given A) | False Negative Rate (FNR) | FN / (TP + FN) | miss rate |
|?|?|accuracy|(TP + TN) / (TP+TN+FP+FN)|?|
|?|?|error rate|(FP + FN) / (TP+TN+FP+FN)|misclassification rate|


**Abbreviations**  
A: Hypothesized Data       
B: Observed Data         
TP: True Positive  
TN: True Negative  
FP: False Positive  
FN: False Negative  
 
^ Note: Sometimes in Bayesian statistics the following terms are used instead:
 
* prior = hypothesis
* posterior = updated hypothesis
* conditional = likelihood
* marginal = model evidence

## Problem 1. Drunk Drivers

imagine that individuals are taking a breathalyzer test with 
* an 8% false positive rate, 
* a 100% true positive rate, 
* our prior belief about drunk driving in the population is 1/1000. 
* What is the probability that a person is drunk after one positive breathalyzer test?

In [104]:
def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):
    prob_non_drunk = 1 - prob_drunk_prior
    numerator = (true_positive_rate*prob_drunk_prior)
    denomenator = ((true_positive_rate*prob_drunk_prior) + (false_positive_rate*prob_non_drunk))
    posterior_probability = (numerator / denomenator)
    return posterior_probability

In [110]:
# Probability that a person is drunk after one breathalyzer test:
posterior = prob_drunk_given_positive(1/1000, .08, 1)
print('{:.4f}'.format(posterior))

0.0124


## Problem 2: Marginal is provided
Solving for the posterior is not too complicated when the marginal probability is provided for you. 

Suppose we have an online website where we sell a gizmo. Consider the case where a website-visitor clicks to expand the gizmo's product description. What is the probability that they will then purchase the gizmo?

Let’s assume some details:  
* 10 percent of site visitors buy the gizmo.  That's the prior: P(buy).
* 7 percent of site visitors that purchased the gizmo also clicked on the description. That's the conditional: P(click|buy).
* 5 percent of site visitors click on the product description. That's the marginal: P(click).
* What percent of site visitors will purchase the gizmo after clicking on the description? That's the posterior: P(buy|click).

Let’s plug what we know into the theorem:
* P(A|B) = P(B|A) * P(A) / P(B)   
* P(buy|click) = P(click|buy) * P(buy) / P(click)

In [90]:
# input the prior, marginal, and conditional.
p_a = 0.10 
p_b = 0.05 
p_b_given_a = 0.07 

In [91]:
# write the formula
def bayes_w_marginal(p_a, p_b, p_b_given_a):
    p_a_given_b = (p_b_given_a * p_a) / p_b
    return p_a_given_b

In [92]:
# plug it in
result = bayes_w_marginal(p_a, p_b, p_b_given_a)
print('P(A|B) = {:.3f}%'.format(result * 100))

P(A|B) = 14.000%


## Problem 3: No marginal provided!
Solving for the posterior is a little harder when the marginal is not provided; most real-world problems fall into this pattern.

Consider the case where we receive an email and the spam detector flags it (i.e., puts it in the spam folder). What is the probability it was actally spam?

Let’s assume some details:  
* 2 percent of the email we receive is spam -- that's the prior: P(Spam). 
* the spam detector is really good and when an email is spam, it flags it 99 percent of the time -- that's the conditional: P(Flagged|Spam).   
* When an email is not spam, it will flag it with a very low rate of 0.1 percent -- that's the fall-out rate: P(Flagged|not Spam).  
* What is the probability that a flagged email is actually spam? -- that's the posterior: P(Spam|Flagged) 

Let’s plug what we know into the theorem:
* P(A|B) = P(B|A) * P(A) / P(B)   
* P(Spam|Flagged) = P(Flagged|Spam) * P(Spam) / P(Flagged)

We don’t know P(B), that is P(Flagged), but we can calculate it as follows:
* P(B) = P(B|A) * P(A) + P(B|not A) * P(not A) 
* P(Flagged) = P(Flagged|Spam) * P(Spam) + P(Flagged|not Spam) * P(not Spam)

In [93]:
# input the prior, conditional, and fall-out rate.
p_a = 0.02 
p_b_given_a = 0.99 
p_b_given_not_a = 0.001 

In [94]:
# write the formula
def bayes_no_marginal(p_a, p_b_given_a, p_b_given_not_a):
    not_a = 1 - p_a
    p_b = p_b_given_a * p_a + p_b_given_not_a * not_a
    p_a_given_b = (p_b_given_a * p_a) / p_b
    return p_a_given_b

In [95]:
# plug it in
posterior = bayes_no_marginal(p_a, p_b_given_a, p_b_given_not_a)
print('P(A|B) = {:.3f}%'.format(posterior * 100))

P(A|B) = 95.284%


## Problem 4. No marginal (again)
What if only two pieces of information are available?

Let’s assume some details:  
* the condition occurs in 2% of the population -- that's the prior: P(sick). 
* when a patient is actually sick, the classifier flags them as sick 72 percent of the time -- that's the conditional: P(Flagged|Sick).   
* when the classifier says they are not sick, this is true 97 percent of the time. That's P(not Flagged | not Sick)
* What is the probability that a flagged patient is actually sick? -- that's the posterior: P(Sick|Flagged).

We don't know the marginal P(B), and we don't know the fall-out rate -- P(Flagged|not Sick) -- but we can calculate them using the formulas:

* P(B) = P(B|A) * P(A) + P(B|not A) * P(not A)
* P(B|not A) = 1 – P(not B|not A)

Which translates to: 
* P(Flagged) = P(Flagged|Sick) * P(Sick) + P(Flagged|not Sick) * P(not Sick)
* P(Flagged|not Sick) = 1 - P(not Flagged|not Sick)

In [96]:
# input the prior, conditional, and P(not B|not A).
p_a = 0.02
p_b_given_a = 0.72
p_not_b_given_not_a = 0.97 

In [97]:
# write the formula:
def bayes_no_marginal_no_fallout(p_a, p_b_given_a, p_not_b_given_not_a):
    not_a = 1 - p_a
    p_b_given_not_a = 1 - p_not_b_given_not_a
    p_b = p_b_given_a * p_a + p_b_given_not_a * not_a
    p_a_given_b = (p_b_given_a * p_a) / p_b
    return p_a_given_b

In [101]:
# plug it in:
posterior = bayes_no_marginal_no_fallout(p_a, p_b_given_a, p_not_b_given_not_a)
print('P(A|B) = {:.3f}%'.format(posterior * 100))

P(A|B) = 32.877%


Sources:  
* http://learningwithdata.com/bayes-primer.html#bayes-primer  
* https://machinelearningmastery.com/intuition-for-bayes-theorem-with-worked-examples/  
* https://www.bayestheorem.net/  
* https://lucdemortier.github.io/articles/16/PerformanceMetrics
* https://towardsdatascience.com/machine-learning-an-error-by-any-other-name-a7760a702c4d
* https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Binary_Diagnostic_Tests-Single_Sample.pdf
* https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/
* https://online.stat.psu.edu/stat507/node/71/